# 2023 IAB Challenge - Text Classification


## About This Challenge

This is the in-class challenge held in Seoul National University.

We will use **techniques learned in class** to classify the given dialog.

사람과 챗봇이 주고 받는 대화가 주어지는데, 사람이 마지막으로 질문의 주체가 속한 카테고리를 예측하는 Text classification 모델을 만들어야합니다.

주어진 데이터의 전처리 과정, 모델 설계 및 학습을 진행합니다.

 **<span style="color:red">단, 이미 학습된 모델을 사용할 수 없습니다. (pre-trained language model 사용 금지)</span>**


아래 스켈레톤 코드를 기반으로 코드를 작성하셔도 되고, 새로 작성하셔도 됩니다.

## **Kaggle 제출 ID: _______**

## 제출 목록 및 기한
* output.csv: **오늘 17:30 까지** kaggle inclass 에 제출
* 본 노트북: **오늘 17:30 까지** ETL 에 제출
* 코드에 대한 보고서: **내일 정오(6/17 11:59 AM)까지** A4 한장 내로 작성하여 ETL 에 제출


## 사전 준비
* 데이터 다운로드
* 라이브러리 import, install

In [ ]:
# 데이터 다운로드
# 아래 코드 실행시 iabc_challenge_20 폴더가 보여야함
!git clone https://github.com/younhyungchae/iab_challenge_23.git

In [ ]:
# 필요한 라이브러리 import, install
!pip install torchtext==0.6.0

import os
import json
import spacy
import spacy.cli
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from torchtext import data, datasets
from torchtext.vocab import Vocab
from collections import Counter

spacy.cli.download("en_core_web_sm")

In [ ]:
# Reproduce 를 하기 위한 셀입니다. 지우지마세요.
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Part 1. Loading Data

이번 챌린지에서 사용하는 데이터를 로드하고, 데이터의 형태를 확인하는 파트입니다.

In [ ]:
DATA_DIR = './iab_challenge_23/'

In [ ]:
# 데이터 로드
def load_data(data_dir, split):
    x = json.load(open(os.path.join(data_dir, split, "logs.json"), 'r'))
    if split != "test":
        y = json.load(open(os.path.join(data_dir, split, "labels.json"), 'r'))
    else: y = []
    return x, y

In [ ]:
X_train, y_train = load_data(DATA_DIR, "train")
X_val, y_val = load_data(DATA_DIR, "val")
X_test, _ = load_data(DATA_DIR, "test")

print("Number of training data:", len(X_train))
print("Number of validation data:", len(X_val))
print("Number of test data:", len(X_test))


### Raw data exploration
* X_train, X_test: User, System 두 speaker 가 주고 받는 대화. json 형태로 구성되어있음.
* y_train, y_test: 마지막 질문이 묻고 있는 대상 (hotel, restaurant, train, taxi)

In [ ]:
X_train[0]

In [ ]:
print("Unique labels:", set([y for y in y_train]))

## Part 2. Preprocess Data & Build Vocab

1. 대화 컨텍스트를 하나의 시퀀스로 변환
2. 라벨을 1-4 class 로 변환
3. torchtext 의 Field 정의하여 vocab 생성
4. Iterator 생성

### 2-1. 대화 컨텍스트를 하나의 시퀀스로 변환
* X_train\[0\]을 보면 dictionary 형태로 각 utterance 가 보여지는데 하나의 문장으로 변환해야함.
* utterance 를 이어붙여 하나의 문장으로 만 때, utterance 개수를 조절하거나, 앞->뒤 혹은 뒤->앞 으로 수정 가능함.
* 또한 화자를 special token (e.g.\<U>, \<S>) 을 정의하여 추가로 표시해줄 수도 있음. 이처럼 해당 태스크만을 위해 사용되는 토큰을 special token 이라고 함
* 아래 예시로 제공된 process_data 는 마지막 utterance 만 추출하도록 작성되어있음
* 제공된 process_data 를 수정하거나 새로 함수를 정의하여 전처리하고, **전처리 방법에 대해 보고서에서 설명해주세요.**


In [ ]:
###########################################################
## 대화 텍스트를 하나의 시퀀스로 전처리하는 함수를 작성하세요.         ##
## 예시로 제공한 함수를 그대로 사용해도 되고, 수정하여 사용해도 됩니다.  ##
###########################################################

special_token = {"U": "<U>", "S": "<S>"}

def process_data(X, window_size=1, use_speaker_tag=False):
    # Arguments 설명
    # window_size: 뒤에서부터 추출하려는 utterance 개수 조절하는 인자.
    #              Default 는 마지막 문장만 추출됨
    # user_speaker_tag: True 일 경우 special token 을 사용하여 화자를 표시.
    #                   위에 정의된 special_token 을 사용할 수도 있고 따로 정의해도 됨.

    X_output = []
    if not use_speaker_tag:
        for log in X:
            input_seq = " ".join([utt["text"] for utt in log[-window_size:]])
            X_output.append(input_seq)
    return X_output

In [ ]:
print("변경 전")
X_train[0]

In [ ]:
###########################################################
## 위에서 정의한 process_data 혹은 본인이 정의한 함수를 사용하여    ##
## 대화 텍스트를 하나의 시퀀스로 전처리하세요.                     ##
###########################################################

X_train = process_data(X_train)
X_val = process_data(X_val)
X_test = process_data(X_test)

In [ ]:
print("변경 후")
X_train[0]

### 2-2. 라벨을 1-4 로 변환

각 라벨을 1-4 로 매핑하는 dictionary 는 테스트 데이터의 output.csv 를 만들 때 사용됩니다.

In [ ]:
LABEL_MAPPING = {
    'hotel': 0,
    'restaurant': 1,
    'taxi': 2,
    'train': 3,
}

In [ ]:
print("변경 전:", y_train[:5])
y_train = [LABEL_MAPPING[y] for y in y_train]
y_val = [LABEL_MAPPING[y] for y in y_val]
print("변경 후:", y_train[:5])

### 2-3. Field 로 정의하여 Vocab 만들기
* torchtext 의 TabularDataset class 사용합니다.
* 이를 위하여 train_data, test_data 를 csv 형태로 저장한 후, TabularDataset 을 사용하여 torchtext Dataset 객체로 생성합니다.

In [ ]:
import pandas as pd
from torchtext.data import TabularDataset

In [ ]:
import pandas as pd
train_data = pd.DataFrame([[x, y] for x, y in zip(X_train, y_train)], columns=["text", "label"])
val_data = pd.DataFrame([[x, y] for x, y in zip(X_val, y_val)], columns=["text", "label"])

In [ ]:
# csv 형태로 변환
print("Train data")
display(train_data.head())
print("\nVal data")
display(val_data.head())

In [ ]:
# csv 로 저장
train_data.to_csv('./iab_challenge_23/train.csv', index=False)
val_data.to_csv('./iab_challenge_23/val.csv', index=False)

In [ ]:
# Field 정의
TEXT = data.Field(use_vocab=True, include_lengths=True)
LABEL = data.LabelField(is_target=True, use_vocab=False)

In [ ]:
train_data, val_data = TabularDataset.splits(
        path='./iab_challenge_23/', train='train.csv', test='val.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [ ]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(val_data)))

* Special token 을 사용했을 경우 vocab 에 해당 토큰을 추가해야합니다.
* 참고: https://torchtext.readthedocs.io/en/latest/vocab.html#module-torchtext.vocab

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 specials=[],
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_
                 )
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

### 2-4. Prepare BucketIterator

- GPU 사용을 위한 Cuda 설정
- Colab 페이지 상단 메뉴>수정>노트설정에서 GPU 사용 설정이 선행되어야 합니다.

In [ ]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

* batch 에 속한 인스턴스의 input 길이가 다르기 때문에 iterator 에서 \<pad> 를 뒤에 붙여서 batch 내 모든 인스턴스의 input 길이가 동일하도록 만듭니다.
* \<pad> 가 적게 들어가도록 sort을 해야하는데 이에 대한 처리를
해야합니다.
* 참고: https://torchtext.readthedocs.io/en/latest/data.html#bucketiterator

In [ ]:
BATCH_SIZE = 32
train_iterator, val_iterator = data.BucketIterator.splits(
    ########################################################################
    ## BucketIterator 를 사용하여 train_iterator, val_iterator 를 정의해주세요. ##
    ## <pad> 를 적게 사용하기 위하여 sorting 을 해야하는 점을 참고하시기 바랍니다.      ##
    #######################################################################
)

## Part 3. Model 구조 만들기

* 본 태스크를 위한 모델을 작성해주세요.
* 예시로 작성된 class 외 새로운 class 를 정의해도 됩니다.
* 하이퍼파라미터도 수정하거나 새로 추가하여 사용하셔도 됩니다.
* **작성 모델 코드에 대한 간단한 설명을 보고서에 작성해주세요.**


In [ ]:
class Model(nn.Module):
    #################################################
    ### 본 태스크를 위한 모델을 만들어주세요.              ###
    ### Arguments 는 본인 모델에 맞게 추가/수정해주세요.   ###
    #################################################
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):

    def forward(self, text):
        return

In [ ]:
#################################################
### 아래 파라미터를 수정하거나, 새로 추가해서 사용하세요.  ###
#################################################

INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = 4
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
N_HEADS = 4
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]


In [ ]:
###################################################
### 예시로 적힌 Arguments 를 앞에 정의한 Model 에 맞게  ###
###    추가/수정하여 model 을 선언하세요.              ###
###################################################

model = Model(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

In [ ]:
# GloVe embedding 을 가져오는 부분입니다. 해당 부분은 수정하지 마세요.
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

## Part 4. Train, Eval, and Test

* 실습 시간에 했던 Sentiment Analysis 와 달리 본 태스크는 label 이 4가지인 Multiclass classification 입니다. 이에 따라 loss 를 Cross Entropy 로 계산합니다.
* cross entropy: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
* accuracy 계산시 각 인스턴스에 대한 label 은 model 로 부터 나온 output 중 가장 큰 값을 가진 label 을 할당합니다. 이때 torch.argmax 사용합니다.
* torch.argmax: https://pytorch.org/docs/stable/generated/torch.argmax.html 중 아래 예시를 참고하세요.


In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
model= model.to(device)
criterion = criterion.to(device)

### 4-1. 학습에 필요한 함수 정의

In [ ]:
def calculate_accuracy(preds, y):
    ##################################################
    ## Accuracy 를 측정하는 함수를 작성해주세요             ##
    ## torch.argmax 를 사용하여 모델에서 출력한 output 중  ##
    ##    가장 큰 값의 인덱스를 label 로 할당합니다.        ##
    #################################################

    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.text[0]).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = calculate_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            predictions = model(batch.text[0]).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = calculate_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


### 4-2. Start training!


In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'trans-model.pt')

    print('Epoch: {:02}'.format(epoch+1))
    print('\tTrain Loss: {:.3f} | Train Acc: {:.2f}%'.format(train_loss, train_acc*100))
    print('\t Val. Loss: {:.3f} |  Val. Acc: {:.2f}%'.format(valid_loss, valid_acc*100))

### 4-3. Predict
* y_train, y_val 전처리 단계에서 string 형태의 카테고리를 1-4 로 label 을 변환했는데, 결과를 출력하기 위해서는 label 을 다시 string 형태의 카테고리로 변환해야합니다.
* 이를 위해 앞서 정의한 LABEL_MAPPING 의 reverse dictionary, LABEL_REV_MAPPING 을 정의합니다.
* 테스트 데이터인 X_test 는 리스트 형태로 데이터가 존재하는데, 리스트 내 각 원소를 순차적으로 1) 토큰화, 2) 모델 입력, 3) 가장 높은 확률을 가진 label 출력, 4) label 을 카테고리로 변환하여 리스트로 리턴하는 함수를 정의합니다.
* 리스트 형태의 결과물을 csv 로 저장한 후, output.csv 를 kaggle inclass 에 제출합니다.

In [ ]:
LABEL_INV_MAPPING = {v:k for k, v in LABEL_MAPPING.items()}
LABEL_INV_MAPPING

In [ ]:
nlp = spacy.load("en_core_web_sm")

def predict(model, X_test):
    all_preds = []
    model.eval()
    for seq in X_test:
        #######################################################################
        ## X_test 의 원소인 seq 를 1) 토큰화, 2) 모델 입력, 3) 가장 높은 확률을 가진    ##
        ##  label 출력, 4) label 을 카테고리로 변환하여, 5) label 리스트에 추가합니다.  ##
        ##  1-3 에 해당하는 작업을 코드로 작성해주세요.                               ##
        ##  3 의 결과물을 max_prob_label 에 저장해주세요.                          ##
        #######################################################################

        # 4, 5번에 대한 코드
        pred = max_prob_label.item()
        all_preds.append(LABEL_INV_MAPPING[int(pred)])
    pd.DataFrame(all_preds, columns=['Category'],
                 index=[i for i in range(len(all_preds))]).to_csv('./iab_challenge_23/output.csv', index_label="Index")
    print("Submit your output.csv into kaggle in-class")
    return all_preds[:5]

In [ ]:
# 이 셀을 실행하면 iab_challenge_20 폴더 안에 output.csv 가 생성됩니다.
# 해당 파일을 다운로드하여 kaggle inclass 에 제출하세요.

predict(model, X_test)

## 주의사항
* 노트북 상단에 kaggle 제출시 사용한 ID 를 기입해주셔야합니다.
* 제출 목록과 기한을 다시 한번 확인해주세요.
  * output.csv: 오늘 17:30 까지 kaggle inclass 에 제출
  * 본 노트북: 오늘 17:30 까지 ETL 에 제출
  * 코드에 대한 보고서: 내일 정오(6/17 11:59 AM)까지 ETL 에 제출